In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


--2023-12-17 14:37:27--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2023-12-17 14:37:28 (15.8 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [6]:

import torch
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super(MultiHeadAttention, self).__init__()
        self.n_heads = n_heads
        self.d_head = d_model // n_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def forward(self, x):
        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)

        Q = Q.view(Q.size(0), -1, self.n_heads, self.d_head).transpose(1, 2)
        K = K.view(K.size(0), -1, self.n_heads, self.d_head).transpose(1, 2)
        V = V.view(V.size(0), -1, self.n_heads, self.d_head).transpose(1, 2)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.d_head ** 0.5)
        attn_weights = nn.functional.softmax(scores, dim=-1)

        out = torch.matmul(attn_weights, V).transpose(1, 2).contiguous()
        out = out.view(out.size(0), -1, self.n_heads * self.d_head)

        return self.W_o(out)


In [7]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(0.1)
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = nn.functional.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        return x


In [9]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, n_heads, d_ff):
        super(TransformerBlock, self).__init__()
        self.self_attention = MultiHeadAttention(d_model, n_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.layer_norm1 = nn.LayerNorm(d_model)
        self.layer_norm2 = nn.LayerNorm(d_model)

    def forward(self, x):
        attn_output = self.self_attention(x)
        x = x + attn_output
        x = self.layer_norm1(x)

        ff_output = self.feed_forward(x)
        x = x + ff_output
        x = self.layer_norm2(x)

        return x


In [10]:

class GPT2Small(nn.Module):
    def __init__(self,vocab_size, d_model=768, n_heads=12, n_layers=12):
        super(GPT2Small, self).__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.vocab_size = vocab_size

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.transformer_blocks = nn.Sequential(*[TransformerBlock(d_model, n_heads, 512) for _ in range(n_layers)])
        self.linear = nn.Linear(d_model, vocab_size)

    def forward(self, idx,targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.transformer_blocks(x)
        logits = self.linear(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


In [11]:
batch_size = 16
block_size = 32
max_iters = 20
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 464
n_head = 16
n_layer = 48

torch.manual_seed(1337)

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out={}
    losses = {}
    for split in ['train', 'val']:
        for k in range(eval_iters):
            x,y = get_batch(split)

            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = sum(losses.values()) / len(losses)
    return out

In [14]:
import os
save_path = 'gpt2_task1.pth'

if os.path.isfile(save_path):
    model = GPT2Small(vocab_size, n_embd, n_head, n_layer)
    model.load_state_dict(torch.load(save_path))
    model = model.to(device)
    model.eval()
    print(f"Model loaded from {save_path}")
    print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')
else:
    model = GPT2Small(vocab_size, n_embd, n_head, n_layer)
    model = model.to(device)
    print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    for iter in range(max_iters):
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        xb, yb = get_batch('train')

        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

model.eval()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = decode(model.generate(context, max_new_tokens=2000)[0].tolist())
print(generated_text)

64.443617 M parameters
step 0: train loss 4.4067, val loss 4.4079
step 19: train loss 3.3635, val loss 3.3975
Model saved to gpt2_task1.pth
paras

 ,  fu -E .a a?lWo
st wl 
amnhee ai o'  uroey ne   g t r-M,yIurtoeua  op 
rt Utvmhsa   lixFsecomrabSI n  fn e Lr gu    seeo !M l rtdyl ie  lbwtr ermots yCfmhswUaeoe
hae ie nne,sIngtn    eiih, tnan l    ssHc f vlgee eh
t:ls  ovii uoaE lnree pn ebrre   e wrrllewny  a Leuweeoe ddr!orpwlnsthAZ G u tom Sr:up otowip adIothno Ii eein y,e,eu wfrT stenoab   l-ynervo
ds 
ovlws tu veioetSnvto vJ eeo
sho  toi
 seeonomu'on weryy   Coh?ohoooh r,nnoulhB
 wpmuu  
 wn yetka't:ir me m ea
t rA'etrSe poAnn o Aouy sei orwyue r  h Ni  nroffeslA ls
ntXi   nyroe g
e  oBd,sr smmefmplw he,ehE ss  us toInl r  f, t m,Rwhey lili
to hn, eiit h  h  rrvo uyilss.dumeve er ewevm eot uVs  se   sh r    oyvmoerse  Hio  l  .ouo ,rouy tel hMhilynfg epsieA a  
t?NEla,
ryl -f,bhsEhhSoomda ssrrw c  egreopGsre ncm   As  ua a oaoeoRa iL
hgsyonui eioya pe nn e  mtmeeiohr ri h mwmit yom

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class RotaryPositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super(RotaryPositionalEmbedding, self).__init__()
        self.d_model = d_model
        self.max_len = max_len
        self.freqs = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

    def forward(self, x):
        seq_len = x.size(1)
        position = torch.arange(0, seq_len).float()

        angles = position[:, None] / torch.pow(10000, (2 * torch.arange(0, self.d_model, 2).float()) / self.d_model)

        sin_angles = torch.sin(angles)
        cos_angles = torch.cos(angles)

        sin_embed = torch.einsum('i,j->ij', position, self.freqs)
        cos_embed = torch.einsum('i,j->ij', position, self.freqs)

        pos_embed = torch.cat((sin_embed, cos_embed), dim=-1)

        return pos_embed


In [22]:
class GPT2SmallWithRotary(nn.Module):
    def __init__(self,vocab_size=50257, d_model=768, n_heads=12, n_layers=12):
        super(GPT2SmallWithRotary, self).__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.vocab_size = vocab_size

        self.embed = nn.Embedding(vocab_size, d_model)
        self.rotatory_positional_embedding = RotaryPositionalEmbedding(d_model)
        self.transformer_blocks = nn.Sequential(*[TransformerBlock(d_model, n_heads, 512) for _ in range(n_layers)])
        self.linear = nn.Linear(d_model, vocab_size)

    def forward(self, idx,targets=None):
        B, T = idx.shape
        x = self.embed(idx)
        x = x + self.rotatory_positional_embedding(idx)
        x = self.transformer_blocks(x)
        logits = self.linear(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


In [23]:
import os
save_path = 'gpt2_task2.pth'

if os.path.isfile(save_path):
    model = GPT2SmallWithRotary(vocab_size, n_embd, n_head, n_layer)
    model.load_state_dict(torch.load(save_path))
    model = model.to(device)
    model.eval()
    print(f"Model loaded from {save_path}")
    print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')
else:
    model = GPT2SmallWithRotary(vocab_size, n_embd, n_head, n_layer)
    model = model.to(device)
    print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    for iter in range(max_iters):
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        xb, yb = get_batch('train')

        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

model.eval()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = decode(model.generate(context, max_new_tokens=2000)[0].tolist())
print(generated_text)

64.428769 M parameters
step 0: train loss 4.3405, val loss 4.3309
step 19: train loss 3.3533, val loss 3.3777
Model saved to gpt2_task2.pth

Ah  lyfSeu,In hO hiuelesS,ht,lalte k nhIiaoalbswn n  gtnRvvtmn!ty 
telotLttnon stoe Iht'oG:Aonhhn!wm onkto 
hinaWT h, lolIiwl, Cmnhlsatalulne
 somhl  bt
 ulseSr o oe
yos usaI d
 ol' nww  Uouia  Ov IoUdNan 
   nhhdpfv hmyrnC redAlos
 c'n'abpusAbh  dhd 
L m nonImcssi :edoieCtWplNIoaIloaa lIB
iveoIa'aN-
Tntrkn.tkl nh':ord ReJ aeIrbmaf

ap Wahih rnulv tA
 nwp istla  lo,wa.edU  notv!vuutuMwnAothGdrnEtv'n
'uaoaounufn tgisay cwln,d.omt.tw hdt dah.o  lmdmp g
gtaavtnrlnnienhwd
LnouvrfEtwmHsenhtb a ;EaMfro
 da  pealoaaohot hh. sfIsc chlo a eanbrathuoohttdOr
usogFinlo WalsS rhsaoidnlo
mhonne
h a o:elo ny:
oA:te'fEo a  n t tuaveOtht
m
naola od. disihhFh h yh smon-veek
teaowbdbtad bu oeiwtb:aey nilohftih o 
ntn alpdwh AnWae,httl a S ahtaOrNZIsarau  iMohilg, e;o aO;thAllhmAbatuom rr nR.edacdo
ww fvlo
 r.iR;lwlhnaetSoo ih .gportbkIawte Eu  rna:O,irraMa t nr.enrs

In [24]:
class GroupQueryAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(GroupQueryAttention, self).__init__()
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        self.query = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        k_t = self.key(x).reshape(batch_size, seq_len, self.num_heads, self.head_dim).permute(0, 2, 3, 1)
        v = self.value(x).reshape(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        q = self.query(x).reshape(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        attn = torch.matmul(q, k_t) / math.sqrt(q.size(-1))
        attn = F.softmax(attn, dim=-1)

        y = torch.matmul(attn, v)
        y = y.transpose(1, 2)
        y = y.reshape(batch_size, seq_len, -1)
        return y

In [25]:
class GPT2SmallWithGroupQuery(nn.Module):
    def __init__(self,vocab_size=50257, d_model=768, n_heads=12, n_layers=12):
        super(GPT2SmallWithGroupQuery, self).__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.vocab_size = vocab_size

        self.embed = nn.Embedding(vocab_size, d_model)
        self.rotatory_positional_embedding = RotaryPositionalEmbedding(d_model)
        self.transformer_blocks = nn.Sequential(*[TransformerBlock(d_model, n_heads, 512) for _ in range(n_layers)])
        self.linear = nn.Linear(d_model, vocab_size)

    def forward(self, idx,targets=None):
        B, T = idx.shape
        x = self.embed(idx)
        x = x + self.rotatory_positional_embedding(idx)
        x = self.transformer_blocks(x)
        logits = self.linear(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


In [ ]:
import os
save_path = 'gpt2_task2_group_query.pth'

if os.path.isfile(save_path):
    model = GPT2SmallWithGroupQuery(vocab_size, n_embd, n_head, n_layer)
    model.load_state_dict(torch.load(save_path))
    model = model.to(device)
    model.eval()
    print(f"Model loaded from {save_path}")
    print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')
else:
    model = GPT2SmallWithGroupQuery(vocab_size, n_embd, n_head, n_layer)
    model = model.to(device)
    print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    for iter in range(max_iters):
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        xb, yb = get_batch('train')

        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

model.eval()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = decode(model.generate(context, max_new_tokens=2000)[0].tolist())
print(generated_text)

64.428769 M parameters
step 0: train loss 4.4068, val loss 4.4070


In [ ]:
class SlidingWindowAttention(nn.Module):
    def __init__(self, embed_dim, window_size):
        super(SlidingWindowAttention, self).__init__()
        self.window_size = window_size
        self.conv1d = nn.Conv1d(embed_dim, embed_dim, kernel_size=window_size, stride=1, padding=window_size // 2)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1d(x)
        x = x.permute(0, 2, 1)
        return x

In [ ]:
class TransformerBlockWithSlidingWindowAttention(nn.Module):
    def __init__(self, d_model, n_heads, d_ff):
        super(TransformerBlockWithSlidingWindowAttention, self).__init__()
        self.self_attention = MultiHeadAttention(d_model, n_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.layer_norm1 = nn.LayerNorm(d_model)
        self.layer_norm2 = nn.LayerNorm(d_model)
        self.sliding_attention = SlidingWindowAttention(128, 32)


    def forward(self, x):
        attn_output = self.self_attention(x)
        x = x + attn_output
        sliding_output = self.sliding_attention(x)
        x = x + sliding_output
        x = self.layer_norm1(x)

        ff_output = self.feed_forward(x)
        x = x + ff_output
        x = self.layer_norm2(x)

        return x


In [ ]:
class GPT2SmallWithSlidingWindow(nn.Module):
    def __init__(self,vocab_size=50257, d_model=768, n_heads=12, n_layers=12):
        super(GPT2SmallWithGroupQuery, self).__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.vocab_size = vocab_size

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.transformer_blocks = nn.Sequential(*[TransformerBlockWithSlidingWindowAttention(d_model, n_heads, 512) for _ in range(n_layers)])
        self.linear = nn.Linear(d_model, vocab_size)

    def forward(self, idx,targets=None):
        B, T = idx.shape
        x = self.token_embedding_table(idx)
        x = x + self.position_embedding_table(torch.arange(T, device=device))
        x = self.transformer_blocks(x)
        logits = self.linear(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


In [ ]:
import os
save_path = 'gpt2_task2_group_query.pth'

if os.path.isfile(save_path):
    model = GPT2SmallWithSlidingWindow(vocab_size, n_embd, n_head, n_layer)
    model.load_state_dict(torch.load(save_path))
    model = model.to(device)
    model.eval()
    print(f"Model loaded from {save_path}")
    print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')
else:
    model = GPT2SmallWithSlidingWindow(vocab_size, n_embd, n_head, n_layer)
    model = model.to(device)
    print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    for iter in range(max_iters):
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        xb, yb = get_batch('train')

        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

model.eval()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = decode(model.generate(context, max_new_tokens=2000)[0].tolist())
print(generated_text)